In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

## Functions

In [2]:
def reduce_indices(indices_A, indices_B, partition):
    
    indices_A, indices_B = np.array(indices_A), np.array(indices_B)
    new_indices_A, new_indices_B = [], []
    
    for i in indices_A:
        new_indices_A.append( i - np.count_nonzero(partition[:i] == 'J') )
    for i in indices_B:
        new_indices_B.append( i - np.count_nonzero(partition[:i] == 'I') )
    
    return new_indices_A, new_indices_B

## Hubbard calculations

In [54]:
K = 4 # number of sites
N_P = K//2 # number of electron pairs

A = gqcpy.AdjacencyMatrix.Cyclic(K)

t = 1.0
U = 3.5
H = gqcpy.HoppingMatrix(A, t, U)
hubbard_hamiltonian = gqcpy.HubbardHamiltonian(H)

onv_basis = gqcpy.SpinResolvedONVBasis(K, N_P, N_P)

solver = gqcpy.EigenproblemSolver.Dense_d()
environment = gqcpy.CIEnvironment.Dense(hubbard_hamiltonian, onv_basis)

wfn = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
coefficients = wfn.coefficients()

## Split the ONV basis into a system and environment

In [56]:
# The system ('I') contains the first and third sites, the environment ('J') the second and fourth sites.
partition = np.array(['I', 'J', 'I', 'J'])

In [57]:
def resolve_into_subsystems(onv_a, I_a, onv_b, I_b):
    
    address = onv_basis.compoundAddress(I_a, I_b)
    print("Address: {}\tONV:\t".format(address), onv_a.__repr__()[::-1], "|", onv_b.__repr__()[::-1], " with partition ", partition)
    
    operator_string_alpha = gqcpy.SpinUnresolvedOperatorString.FromONV(onv_a)
    operator_string_beta = gqcpy.SpinUnresolvedOperatorString.FromONV(onv_b)
    
    # The number of spinors in the system and environment depends on the partition.
    K_system = np.count_nonzero(partition == 'I')
    K_environment = np.count_nonzero(partition== 'J')
    
    # We only need the occupied indices for the alpha and beta operator string to determine the phase factor of the ONV.
    alpha_partition = partition[operator_string_alpha.operatorIndices()]
    beta_partition = partition[operator_string_beta.operatorIndices()]
    
    # Decompose the operator string into a system and environment for each spin.
    system_alpha, environment_alpha = operator_string_alpha.resolveIntoSubsystems(alpha_partition)
    system_beta, environment_beta = operator_string_beta.resolveIntoSubsystems(beta_partition)
    
    # In order to make the ONVs for the system and environment, we must take the correct indices into account.
    system_alpha_indices, environment_alpha_indices = reduce_indices(system_alpha.operatorIndices(), environment_alpha.operatorIndices(), partition)
    onv_system_alpha = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(system_alpha_indices, K_system)
    onv_environment_alpha = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(environment_alpha_indices, K_environment)
    
    system_beta_indices, environment_beta_indices = reduce_indices(system_beta.operatorIndices(), environment_beta.operatorIndices(), partition)
    onv_system_beta = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(system_beta_indices, K_system)
    onv_environment_beta = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(environment_beta_indices, K_environment)
    
    onv_system = gqcpy.SpinResolvedONV(onv_system_alpha, onv_system_beta)
    onv_environment = gqcpy.SpinResolvedONV(onv_environment_alpha, onv_environment_beta)
    # Note: read ONVs from left to right due to reversing the bitstring representation.
    print("The spin-resolved ONV of the system is: {}|{} with phase factor {}.".format(onv_system_alpha.__repr__()[::-1], onv_system_beta.__repr__()[::-1], system_alpha.phaseFactor()))
    print("The spin-resolved ONV of the environment is: {}|{} with phase factor {}.\n".format(onv_environment_alpha.__repr__()[::-1], onv_environment_beta.__repr__()[::-1], system_beta.phaseFactor()))
    
    # Assign a correct phase factor to the coefficient of the ONV after resolving.
    coefficients[address] *= system_alpha.phaseFactor() * system_beta.phaseFactor()
    
    #print("The alpha system consists of sites: {} and has phase factor {}.".format(system_alpha.operatorIndices(), system_alpha.phaseFactor()))
    #print("The alpha environment consists of sites: {} and has phase factor {}.".format(environment_alpha.operatorIndices(), environment_alpha.phaseFactor()))
    #print("The beta system consists of sites: {} and has phase factor {}.".format(system_beta.operatorIndices(), system_beta.phaseFactor()))
    #print("The beta environment consists of sites: {} and has phase factor {}.".format(environment_beta.operatorIndices(), environment_beta.phaseFactor()))
    
    #print("Alpha system: correction of indices {} to {}.".format(system_alpha.operatorIndices(), system_alpha_indices))
    #print("Beta system: correction of indices {} to {}.".format(system_beta.operatorIndices(), system_beta_indices))
    #print("Alpha environment: correction of indices {} to {}.".format(environment_alpha.operatorIndices(), environment_alpha_indices))
    #print("Beta environment: correction of indices {} to {}.".format(environment_beta.operatorIndices(), environment_beta_indices))
    
    
    system_onvs.append(onv_system)
    environment_onvs.append(onv_environment)
    
    # The environment ONV basis should not contain duplicate ONVs.
    if onv_environment.__repr__() not in j_environment_strings:
        j_environment_onvs.append(onv_environment)
        j_environment_strings.append(onv_environment.__repr__())

In [64]:
system_onvs, environment_onvs, j_environment_onvs = [], [], []
system_strings, environment_strings, j_environment_strings = [], [], []

onv_basis.forEach(resolve_into_subsystems)

Address: 0	ONV:	 1100 | 1100  with partition  ['I' 'J' 'I' 'J']
The spin-resolved ONV of the system is: 10|10 with phase factor 1.
The spin-resolved ONV of the environment is: 10|10 with phase factor 1.

Address: 1	ONV:	 1100 | 1010  with partition  ['I' 'J' 'I' 'J']
The spin-resolved ONV of the system is: 10|11 with phase factor 1.
The spin-resolved ONV of the environment is: 10|00 with phase factor 1.

Address: 2	ONV:	 1100 | 0110  with partition  ['I' 'J' 'I' 'J']
The spin-resolved ONV of the system is: 10|01 with phase factor 1.
The spin-resolved ONV of the environment is: 10|10 with phase factor -1.

Address: 3	ONV:	 1100 | 1001  with partition  ['I' 'J' 'I' 'J']
The spin-resolved ONV of the system is: 10|10 with phase factor 1.
The spin-resolved ONV of the environment is: 10|01 with phase factor 1.

Address: 4	ONV:	 1100 | 0101  with partition  ['I' 'J' 'I' 'J']
The spin-resolved ONV of the system is: 10|00 with phase factor 1.
The spin-resolved ONV of the environment is: 10|11 w

In [65]:
# "Old" coefficients.
wfn.coefficients()

array([-5.854e-02, -1.354e-01,  5.605e-17, -5.581e-17,  1.354e-01,  2.403e-01, -1.354e-01, -3.404e-16,  1.354e-01,
        1.354e-01,  4.806e-01,  1.354e-01, -5.017e-17,  1.354e-01,  5.854e-02,  2.403e-01,  1.354e-01,  3.815e-17,
       -4.738e-17,  1.354e-01,  2.403e-01,  5.854e-02,  1.354e-01,  3.225e-17,  1.354e-01,  4.806e-01,  1.354e-01,
        1.354e-01, -3.876e-16, -1.354e-01,  2.403e-01,  1.354e-01, -3.984e-17, -4.177e-17, -1.354e-01, -5.854e-02])

In [66]:
# Coefficients with the correct phase factor after resolving into subsystems.
coefficients

array([-5.854e-02, -1.354e-01,  5.605e-17, -5.581e-17,  1.354e-01,  2.403e-01, -1.354e-01, -3.404e-16,  1.354e-01,
        1.354e-01,  4.806e-01,  1.354e-01, -5.017e-17,  1.354e-01,  5.854e-02,  2.403e-01,  1.354e-01,  3.815e-17,
       -4.738e-17,  1.354e-01,  2.403e-01,  5.854e-02,  1.354e-01,  3.225e-17,  1.354e-01,  4.806e-01,  1.354e-01,
        1.354e-01, -3.876e-16, -1.354e-01,  2.403e-01,  1.354e-01, -3.984e-17, -4.177e-17, -1.354e-01, -5.854e-02])

In [74]:
K_system = np.count_nonzero(partition == 'I')
K_environment = np.count_nonzero(partition== 'J')
N_system = 1

# "01|10"
onv_n_bra = gqcpy.SpinResolvedONV(gqcpy.SpinUnresolvedONV(K_system, N_system, 1), gqcpy.SpinUnresolvedONV(K_system, N_system, 2))
# "10|01"
onv_n_ket = gqcpy.SpinResolvedONV(gqcpy.SpinUnresolvedONV(K_system, N_system, 2), gqcpy.SpinUnresolvedONV(K_system, N_system, 1))

In [80]:
matrix_element = 0.0

for p,onv_system_ket in enumerate(system_onvs):
    for q,onv_system_bra in enumerate(system_onvs):
    
        if (onv_system_ket == onv_n_bra) and (onv_system_bra == onv_n_ket):
        
            for j,onv_j in enumerate(j_environment_onvs):
                
                print("< {} | < {} | | {} >| {} > < {} |< {} | | {} > | {} >"
                          .format(j_environment_onvs[r], 
                                  onv_n_bra, onv_system_ket, environment_onvs[p],
                                  environment_onvs[q], onv_system_bra, onv_n_ket, 
                                  j_environment_onvs[r]), end="\t")
                
                if (onv_j == environment_onvs[p]) and (onv_j == environment_onvs[q]):
                    matrix_element += coefficients[p] * coefficients[q]
                    print("-> ", np.round(coefficients[p] * coefficients[q], 6), " // ({},{})".format(p, q))
                else:
                    print()
        
print(matrix_element)

< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	->  -0.0  // (2,12)
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01|01 > < 01|01 |< 10|01 | | 10|01 > | 11|11 >	
< 11|11 | < 01|10 | | 01|10 >| 01